<a href="https://colab.research.google.com/github/Marcu-Loreto/UFG_Estudo_caso_2025/blob/main/UFG_RAG_Aula02_Sala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain-community
!pip install langchain
!pip install torch
!pip install sentence-transformers
!pip install faiss-cpu
!pip install -U langchain-google-genai

  Using cached langchain_community-0.3.31-py3-none-any.whl.metadata (3.0 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain_community-0.3.31-py3-none-any.whl (2.5 MB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached marshmallow-3.26.1-py3-none-any.whl (50 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Using cached mypy_extensions-1.1.0-py3-none-any.whl (5.0 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently

In [ ]:
# -*- coding: utf-8 -*-
!pip install requests beautifulsoup4 pandas

In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=4dd83f756562cfecc8af38977d73d38dabf5c32c50dbc375f07435455fab8b73
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


### Saudações a todos,
#### Para esse projeto é importante ressaltar que será necessário obter uma chave gratuita do Gemini, abaixo está o passo-a-passo:
    1. Acesse o site oficial do Gemini: https://aistudio.google.com/app/apikey

    2. Crie uma conta gratuita ou faça login com uma conta Google.

    3. Vá para a seção API Keys ou Credenciais do painel do usuário.

    4. Clique em Criar nova chave ou Generate API Key.

    5. Copie a chave gerada e guarde em um local seguro.


#### RAG (Geração com Recuperação de Informações) é uma técnica que combina:

Modelos de linguagem (LLMs) – como GPT, BERT, etc., capazes de gerar texto ou responder perguntas.

Base de conhecimento externa – documentos, PDFs, CSVs, sites, etc.

Como funciona:

* Em vez do modelo depender apenas do conhecimento interno, ele consulta documentos relevantes em uma base externa.

* O processo típico inclui:

* Pré-processamento: dividir documentos em chunks, criar embeddings.

* Armazenamento vetorial: usar FAISS ou outro índice para buscar chunks semanticamente relevantes.

* Consulta: dado um prompt/pergunta, buscar os chunks mais próximos.

* Geração: o LLM gera uma resposta baseada nos chunks recuperados.

Vantagens do RAG:

* Respostas mais precisas e atualizadas.

* Reduz o risco de "alucinação" do modelo.

* Permite lidar com bases de conhecimento muito grandes sem sobrecarregar o modelo.

In [ ]:
from IPython.display import Image, display
url = 'https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_workflow.png'
display(Image(url=url))


In [ ]:
import pandas as pd #Nossa base de dados é um dataset com todos os presidentes da república desde Deodoro da Fonseca até Lula no seu 3º governo. Todos os textos foram raspados da wikipedia

caminho_arquivo = 'https://raw.githubusercontent.com/sicdavyd/dados/refs/heads/main/presidentes_brasil_definitivo.csv'

In [ ]:
dados = pd.read_csv(caminho_arquivo)

In [ ]:
dados[:10]

,Unnamed: 0,nome,historia
0,0,Primeira República,Ditadura militar ( 1889 – 1894 )\nOligarquia (...
1,1,Deodoro da Fonseca,Manuel Deodoro da Fonseca ( Alagoas da Lagoa d...
2,2,Floriano Peixoto,"Floriano Vieira Peixoto ( Maceió , 30 de abril..."
3,3,Prudente de Moraes,Prudente José de Morais Barros ( Distrito do V...
4,4,Campos Salles,"Manoel Ferraz de Campos Sales ( Campinas , 1..."
5,5,Rodrigues Alves,Francisco de Paula Rodrigues Alves ( Guarating...
6,6,Affonso Penna,Afonso Augusto Moreira Pena ( Santa Bárbara ...
7,7,Nilo Peçanha,Nilo Procópio Peçanha GCSE ( Campos dos Goytac...
8,8,Hermes da Fonseca,"Hermes Rodrigues da Fonseca ( São Gabriel , 12..."
9,9,Wenceslau Braz,Wenceslau Braz Pereira Gomes ONM ( São Cae...


In [ ]:
from langchain.docstore.document import Document as LangchainDocument # Essa classe representa cada documento, com conteúdo e metadados

# cria uma lista de documentos a partir de um DataFrame chamado 'dados'
BASE_DADOS_BRUTA = [
    # P/ cada linha do dataframe, cria um documento do langchain, page_content armazena 'historia', metadata armazena outras
    LangchainDocument(page_content=row["historia"], metadata={"nome": row["nome"]})
    for _, row in dados.iterrows()
]


In [ ]:
BASE_DADOS_BRUTA[11:12]

[Document(metadata={'nome': 'Epitacio Pessôa'}, page_content='Epitácio Lindolfo da Silva Pessoa   ( Umbuzeiro , 23 de maio de 1865 – Petrópolis , 13 de fevereiro de 1942 ) foi um magistrado , diplomata , professor universitário , jurista e político brasileiro , filiado ao Partido Republicano Mineiro . Foi o 11.º presidente do Brasil entre os anos de 1919 a 1922 , tendo o seu governo marcado por revoltas militares que acabariam na Revolução de 1930 , a qual levou Getúlio Vargas ao governo central.\nAnteriormente, foi deputado federal em duas oportunidades, ministro da Justiça , do Supremo Tribunal Federal , procurador-geral da República , senador três vezes, chefe da delegação brasileira junto à Conferência de Versalhes e juiz da então Corte Permanente de Justiça Internacional .\nNasceu na Paraíba , onde os pais morreram de varíola quando tinha sete anos de idade. Foi educado e criado pelo tio, Henrique de Lucena , o Barão de Lucena, então governador de Pernambuco . Era primo em 2º grau

In [ ]:
# Importa o utilitário para dividir textos longos em pedaços menores chunks;
from langchain.text_splitter import RecursiveCharacterTextSplitter

separador_texto = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # número máximo de caracteres em cada chunk (definido arbitrariamente)
    chunk_overlap=150,  # número de caracteres que se sobrepõem entre chunks consecutivos
    add_start_index=True,  # se True, adiciona no metadado o índice inicial do chunk dentro do documento original
    strip_whitespace=True,  # se True, remove espaços em branco no início e fim de cada chunk

)

# lista que armazenará os documentos após a divisão em chunks
documentos_processados = []

# Para cada documento da base de conhecimento original 'BASE_DADOS_BRUTA',
# aplicamos a divisão em chunks e acumulamos os resultados.
for doc in BASE_DADOS_BRUTA:
    documentos_processados += separador_texto.split_documents([doc])

### Ao rodar a célula abaixo provavelmente aparecerá uma mensagem pedindo para liberar acesso ao HF TOKEN, permita!

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import torch
#from langchain_huggingface import HuggingFaceEmbeddings

# detecta se há GPU disponível cuda, caso contrário usa cpu
dispositivo = "cuda" if torch.cuda.is_available() else "cpu"

# criamos o modelo de embeddings multilíngue (aceita ptbr)
# modelo escolhido é leve e eficiente para comparação semântica de frases.
modelo_embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",  # modelo usado
    model_kwargs={"device": dispositivo},         # executa na GPU se disponível
    encode_kwargs={"normalize_embeddings": True},  # normaliza vetores para similaridade por cosseno
)

# Criamos a base vetorial usando FAISS
# Cada documento previamente dividido (documentos_processados) é convertido em embedding
# e armazenado em uma estrutura de busca eficiente.
vector_db = FAISS.from_documents(
    documentos_processados,  # conjunto de chunks processados
    modelo_embedding,        # modelo de embeddings em português
)


/tmp/ipython-input-3185217437.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  modelo_embedding = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# definir a pergunta que queremos testar
pergunta = "Lula" # Nesse teste a resposta correta está no chunk 6

''' buscar os chunks mais relevantes na base vetorial
    k=8 retorna os 8 chunks mais próximos da pergunta
    você pode testar com valores diferentes de k para ver como muda o resultado '''
resultado = vector_db.similarity_search(pergunta, k=8)

for i, doc in enumerate(resultado):
    print(f"Chunk {i+1}:")
    print(doc.page_content[:500])
    print(f'--' * 250)


Chunk 1:
Dilma passou a integrar a caravana de Lula no Nordeste brasileiro , tornando-se o principal cabo eleitoral de seu antecessor, que foi pré-candidato à presidência da República em 2018. A caravana foi marcada pela presença de apoiadores ilustres do Partido dos Trabalhadores e por críticas veementes de Lula e Dilma ao Poder Judiciário e ao governo Michel Temer , que "não tem o menor compromisso com o povo brasileiro", segundo Dilma.   Foi uma das fundadoras do Grupo de Puebla ,   entidade apontada 
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Abaixo você deve inserir sua Google API Key entre aspas, simples ou duplas.

In [ ]:

from langchain.chat_models import init_chat_model
import os
from google.colab import userdata
from langchain.chat_models import init_chat_model
# Substitua 'SUA_CHAVE' pela chave real
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [ ]:
from langchain.chains import RetrievalQA

# criar o retriever a partir da base FAISS
# ele vai buscar os chunks mais relevantes na base vetorial
retriever = vector_db.as_retriever(
    search_type= "mmr",  # tipo de busca: "similarity" (mais comum - similaridade do cosseno) ou "mmr" (para maior diversidade) brinque com isso!
    search_kwargs={"k": 5}     # número de chunks que serão retornados
)

# montar o pipeline RAG usando o LLM (ex.: Gemini) e o retriever
# o RetrievalQA conecta a busca de chunks com a geração da resposta pelo modelo, essencial
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,                # modelo de linguagem que vai gerar a resposta
    retriever=retriever,    # componente que busca no FAISS
    chain_type= "refine" #"map_reduce" #"stuff"      # "stuff" concatena todos os chunks encontrados de forma simples. Outra opcoes: map_reduce e "refine"
)

# testar o pipeline com uma pergunta
pergunta = "Quem foi Getulio Vargas"
resposta = qa_chain.run(pergunta)

# exibir resultado
print("Pergunta:", pergunta)
print("Resposta:", resposta)


Pergunta: Quem foi Getulio Vargas
Resposta: Getúlio Vargas foi uma das figuras mais influentes e complexas da história política brasileira, cujo legado ainda ressoa fortemente no país.

Nasceu em 19 de abril de 1882, no município de São Borja, no interior do Rio Grande do Sul. Era filho de Manuel do Nascimento Vargas e de Cândida Francisca Dornelles Vargas. Curiosamente, ele alterou alguns documentos na juventude para fazer constar o ano de nascimento como 1883, um fato que só foi descoberto nas comemorações do seu centenário, ao verificar os livros de registros de batismos. Em 1950, antes de uma campanha eleitoral, ele chegou a corrigir repórteres, afirmando ter nascido em 1883.

Na política, Getúlio Vargas apoiou a Aliança Liberal e foi um dos mentores intelectuais da Revolução de 1930, que o levou ao poder. Como presidente, ele instaurou o Estado Novo no Brasil através do golpe de 10 de novembro de 1937, período de forte centralização de poder e modernização do país. Seu governo foi

## Vamos fazer o mesmo processo feito anteriormente, porém, com Prompt customizado que utiliza a técnica few-shot

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# prompt inicial (instrução de como o modelo deve responder)
prompt_inicial = PromptTemplate(
    template="""
Você é um historiador acadêmico especialista em história do Brasil, conhecido por sua precisão e objetividade. Sua tarefa é responder perguntas baseando-se *exclusivamente* nas informações fornecidas no contexto. Siga as instruções e os exemplos de comportamento abaixo.

**Instruções rigorosas:**
1.  **Atenha-se ao Contexto:** Extraia e sintetize a resposta apenas do texto fornecido em "Contexto". Não utilize conhecimento prévio ou informações externas.
2.  **Seja Direto e Conciso:** Formule uma resposta clara, objetiva e sem rodeios.
3.  **Regra de Recusa:** Se a resposta não estiver no contexto, responda exatamente: "A informação para responder a esta pergunta não foi encontrada no contexto fornecido."

---
**Exemplo 1 (Resposta Encontrada):**

**Contexto:** Durante a Era Vargas, especificamente no período do Estado Novo, o presidente Getúlio Vargas promoveu uma significativa centralização do poder. Uma de suas ações mais notáveis foi a criação da Consolidação das Leis do Trabalho (CLT) em 1º de maio de 1943, que unificou a legislação trabalhista no país.
**Pergunta:** Qual foi a principal legislação trabalhista criada por Getúlio Vargas e em que ano?
**Resposta:** Segundo o contexto, a principal legislação trabalhista criada por Getúlio Vargas foi a Consolidação das Leis do Trabalho (CLT), em 1943.

---
**Exemplo 2 (Resposta NÃO Encontrada):**

**Contexto:** O governo de Juscelino Kubitschek (1956-1961) foi marcado pelo seu ambicioso Plano de Metas, cujo lema era "cinquenta anos em cinco". O projeto mais emblemático de seu mandato foi a construção de uma nova capital federal, Brasília, inaugurada em 21 de abril de 1960.
**Pergunta:** Qual era o nome da esposa de Juscelino Kubitschek?
**Resposta:** A informação para responder a esta pergunta não foi encontrada no contexto fornecido.

---
**Exemplo 3 (Síntese de Informação):**

**Contexto:** A presidência de Fernando Henrique Cardoso (FHC) teve como pilar a estabilização econômica do país, consolidando o Plano Real, que controlou a hiperinflação. Além da política monetária, seu governo também implementou um vasto programa de privatizações de empresas estatais em setores como telecomunicações e mineração.
**Pergunta:** Quais foram as duas principais frentes de atuação do governo FHC mencionadas no texto?
**Resposta:** De acordo com o texto, as duas principais frentes de atuação do governo FHC foram a consolidação do Plano Real para controlar a inflação e a implementação de um programa de privatizações de empresas estatais.
---

**Sua Tarefa:**

**Contexto:**
{context}

**Pergunta:**
{question}

**Resposta:**
""",
    input_variables=["context", "question"]
)

## Execução do Prompt customizado que utiliza a técnica few-shot

In [ ]:
# criar o retriever a partir da base FAISS
retriever_ = vector_db.as_retriever(
    search_type="similarity", #afeta como os chunks são buscados; outra opção: mmr
    search_kwargs={"k": 30} #top k entidades retornadas pelo retriever
)

# montar o pipeline RAG com prompt inicial
qa_chain_ = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever_,
    chain_type="stuff", # define como os chunks são combinados antes do modelo gerar a resposta
    chain_type_kwargs={"prompt": prompt_inicial},  # parametros extras, como: prompt_template

)

In [ ]:
# testar o pipeline
pergunta_ = "café filho"
resposta_ = qa_chain_.run(pergunta_)

print("Pergunta:", pergunta_)
print("Resposta:", resposta_)

Pergunta: café filho
Resposta: De acordo com o contexto, João Fernandes Campos Café Filho foi um advogado e político brasileiro, tendo sido o 18º presidente do Brasil entre 24 de agosto de 1954 e 8 de novembro de 1955, e o 13º vice-presidente do país entre 1951 e 1954, período em que também assumiu a função de presidente do Senado Federal.

Nascido em Natal, Rio Grande do Norte, em 3 de fevereiro de 1899, foi o único potiguar e o primeiro protestante a ocupar a presidência, além de ser o primeiro presidente nascido após a Proclamação da República. Em sua juventude, trabalhou como jornalista e atuou como goleiro do Alecrim Futebol Clube, sendo o único presidente a ter sido jogador competitivo de futebol adulto.

Sua candidatura à vice-presidência em 1950 foi uma imposição de Ademar de Barros para apoiar Getúlio Vargas, apesar de Vargas resistir devido a militares e à Igreja Católica o considerarem esquerdista. Ele foi eleito vice-presidente e também reeleito deputado federal pelo Rio Gr